In [1]:
import os
import glob
import numpy as np
import pandas as pd
import pickle
import joblib
import seaborn as sns
import networkx as nx
from graph_utils import get_graph_features
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
data_path = '../data/depression/'
folders_list = sorted(os.listdir(data_path))

In [3]:
corr_matrix_list = []

for folder in folders_list:
    df_path = data_path + folder
    corr_matrix2append = pd.read_csv(df_path, header=None).values
    corr_matrix_list.append(corr_matrix2append)

In [4]:
y = joblib.load('../data/y_d_nod.pkl')
y_all = joblib.load('../data/y_all.pkl')

In [5]:
graph_features_list = []
y_list = []

for A, y_value in zip(corr_matrix_list, y):
    graph_features = get_graph_features(A, border_value=0.22, min_graph_size=3,
                                        max_graph_size=1500, max_number_of_edges=700)
    if graph_features is not None:
        graph_features_list.append(graph_features)
        y_list.append(y_value)

n_edges=37.0
37 21 37
n_edges=10.0
10 2 10
n_edges=16.0
16 1 16
n_edges=7.0
7 0 7
n_edges=65.0
65 57 65
n_edges=22.0
22 5 22
n_edges=125.0
125 167 125
n_edges=12.0
12 2 12
n_edges=58.0
58 22 58
n_edges=6.0
6 0 6
n_edges=34.0
34 13 34
n_edges=45.0
45 31 45
n_edges=92.0
92 103 92
n_edges=174.0
174 265 174
n_edges=66.0
66 28 66
n_edges=9.0
9 0 9
n_edges=21.0
21 8 21
n_edges=18.0
18 3 18
n_edges=10.0
10 5 10
n_edges=11.0
11 1 11
n_edges=15.0
15 4 15
n_edges=32.0
32 8 32
n_edges=22.0
22 3 22
n_edges=41.0
41 15 41
n_edges=78.0
78 111 78
n_edges=54.0
54 42 54
n_edges=19.0
19 2 19
n_edges=45.0
45 10 45
n_edges=19.0
19 2 19
n_edges=56.0
56 25 56
n_edges=45.0
45 20 45
n_edges=50.0
50 21 50
n_edges=22.0
22 5 22
n_edges=16.0
16 2 16
n_edges=31.0
31 11 31
n_edges=36.0
36 15 36
n_edges=33.0
33 19 33
n_edges=42.0
42 19 42
n_edges=30.0
30 4 30
n_edges=12.0
12 0 12
n_edges=17.0
17 1 17
n_edges=14.0
14 0 14
n_edges=11.0
11 1 11
n_edges=24.0
24 4 24
n_edges=24.0
24 4 24
n_edges=59.0
59 47 59
n_edges=40.0

In [6]:
joblib.dump(graph_features_list, 'graph_features_list_depression')
joblib.dump(y_list, 'y_list_depression')

['y_list_depression']

In [7]:
aggregated_graph_features_list = []

for graph_features in graph_features_list:
    aggregated_graph_features_dict = {}
    for feature_name, feature_values in graph_features.items():
        aggregated_feature_name = feature_name + '_median'
        aggregated_graph_features_dict[aggregated_feature_name] = np.median(feature_values)
        
#         aggregated_feature_name = feature_name + '_mean'
#         aggregated_graph_features_dict[aggregated_feature_name] = np.mean(feature_values)
    
    aggregated_graph_features_list.append(aggregated_graph_features_dict)

In [8]:
df = pd.DataFrame(aggregated_graph_features_list)
df = df.fillna(0)

In [9]:
print(df.shape)
df.head()

(98, 32)


,AvgNodeDegree_G_2click_1simplex_median,AvgNodeDegree_G_3click_1simplex_median,AvgNodeDegree_G_3click_2simplex_median,AvgNodeDegree_initial_median,Betweenness_G_2click_1simplex_median,Betweenness_G_3click_1simplex_median,Betweenness_G_3click_2simplex_median,Betweenness_initial_median,Closeness_G_2click_1simplex_median,Closeness_G_3click_1simplex_median,...,GlobalEfficiency_G_3click_2simplex_median,GlobalEfficiency_initial_median,LocalEfficiency_G_2click_1simplex_median,LocalEfficiency_G_3click_1simplex_median,LocalEfficiency_G_3click_2simplex_median,LocalEfficiency_initial_median,n_2click_1simplex_median,n_3click_1simplex_median,n_3click_2simplex_median,n_edges_median
0,9.000,5.000000,7.0,0.0,0.006349,0.001587,0.027558,0.0,0.363363,0.160714,...,0.529365,0.009468,0.735689,0.592235,0.669724,0.102889,37.0,37.0,21.0,37.0
1,3.575,1.000000,0.0,0.0,0.004630,0.000000,0.000000,0.0,0.396825,0.111111,...,0.000000,0.001768,0.513333,0.233333,0.000000,0.027778,10.0,10.0,2.0,10.0
2,0.000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.003136,0.232639,0.000000,0.000000,0.014245,16.0,16.0,1.0,16.0
3,0.000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.001032,0.000000,0.000000,0.000000,0.000000,7.0,7.0,0.0,7.0
4,9.875,3.222222,12.5,0.0,0.001538,0.000000,0.012879,0.0,0.281250,0.148053,...,0.401650,0.013631,0.671903,0.155989,0.668472,0.167727,65.0,65.0,57.0,65.0


In [10]:
df.to_csv('df.csv', index=False)

In [11]:
# df_with_target = df.copy()
# df_with_target['target'] = y_list

# df_corr = df_with_target.corr()

# feature_names = df_corr.columns

# plt.figure(figsize=(12, 10))
# sns.heatmap(df_corr.values, cmap='coolwarm', square=True, vmin=-1, vmax=1, cbar_kws={"shrink": .7})
# # plt.xlabel(feature_names)
# plt.yticks(np.arange(len(feature_names)) + 0.5, feature_names, rotation=0, va="center")
# plt.xticks(np.arange(len(feature_names)) + 0.5, feature_names, rotation=45, rotation_mode="anchor", ha="right")
# plt.savefig('../pic/heatmap.png')
# plt.tight_layout()

In [12]:
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, Lasso
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import RepeatedKFold
from sklearn.neighbors import KNeighborsClassifier

In [13]:
target = np.array(y_list)
ss = StandardScaler()

allowed_suffixes_list_initial = ['initial']
allowed_suffixes_list_2_click = ['initial', '2click_1simplex']
allowed_suffixes_list_3_click = ['initial', '3click_1simplex', '3click_2simplex']

columns_initial = [column for column in df.columns if any(suffix in column for suffix in allowed_suffixes_list_initial)]
columns_2click = [column for column in df.columns if any(suffix in column for suffix in allowed_suffixes_list_2_click)]
columns_3click = [column for column in df.columns if any(suffix in column for suffix in allowed_suffixes_list_3_click)]
# columns_all = list(df.columns)

In [14]:
rk = RepeatedKFold(n_splits=5, n_repeats=10)

# lasso = Lasso(alpha=0.01)
rf = RandomForestClassifier(n_estimators=100, max_depth=3, n_jobs=-1)
lr = LogisticRegression(solver='lbfgs', max_iter=10000, n_jobs=-1)
svm = SVC(gamma='scale', kernel='rbf')
knn = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)

methods_list = [lr, rf, svm, knn]

In [36]:
results_list = []

for method in methods_list:
    method_name = method.__class__.__name__
    result = {
        'method_name': method_name,
    }

    for columns_descryption, columns_list in zip(
        ['initial', '2-level features', '3-level features'],
        [columns_initial, columns_2click, columns_3click],
    ):
        train_normed = ss.fit_transform(df[columns_list].values)
        scoring_method = 'roc_auc'
        method_scores = cross_val_score(method, X=train_normed, y=target, cv=rk, scoring=scoring_method)
        method_score_mean = method_scores.mean()
        method_score_std = method_scores.std()
        result4columns = {
            f'{columns_descryption}_mean': method_score_mean,
            f'{columns_descryption}_std': method_score_std,
        }
        result.update(result4columns)

    results_list.append(result)#, name=method_name))

df_results = pd.DataFrame(results_list)        

In [37]:
column_order = ['method_name', 'initial_mean','2-level features_mean', '3-level features_mean']#, '2-level features_std', '3-level features_std']
df_results[column_order].round(3)

,method_name,initial_mean,2-level features_mean,3-level features_mean
0,LogisticRegression,0.597,0.590,0.516
1,RandomForestClassifier,0.552,0.527,0.541
2,SVC,0.609,0.563,0.575
3,KNeighborsClassifier,0.492,0.486,0.516


### Feature selection

In [17]:
lasso = Lasso(alpha=0.01)

In [38]:
best_features_dict = {}

for columns_descryption, columns_list in zip(
    ['initial', '2-level features', '3-level features'],
    [columns_initial, columns_2click, columns_3click],
):
    train_normed = ss.fit_transform(df[columns_list].values)

    lasso.fit(train_normed, target)
    useful_features_nums = np.nonzero(lasso.coef_)[0]
    n_features = len(useful_features_nums)
    print(columns_descryption, n_features)
    best_features_dict[columns_descryption] = np.array(columns_list)[useful_features_nums]

initial 3
2-level features 7
3-level features 11


In [39]:
results_list = []

for method in methods_list:
    method_name = method.__class__.__name__
    result = {
        'method_name': method_name,
    }

    for columns_descryption, best_features in best_features_dict.items():
        train_normed = ss.fit_transform(df[best_features].values)
        scoring_method = 'roc_auc'
        method_scores = cross_val_score(method, X=train_normed, y=target, cv=rk, scoring=scoring_method)
        method_score_mean = method_scores.mean()
        method_score_std = method_scores.std()
        result4columns = {
            f'{columns_descryption}_mean': method_score_mean,
            f'{columns_descryption}_std': method_score_std,
        }
        result.update(result4columns)

    results_list.append(result)#, name=method_name))

df_results = pd.DataFrame(results_list)

In [40]:
df_results[column_order].round(3)

,method_name,initial_mean,2-level features_mean,3-level features_mean
0,LogisticRegression,0.632,0.610,0.594
1,RandomForestClassifier,0.575,0.560,0.584
2,SVC,0.607,0.604,0.589
3,KNeighborsClassifier,0.503,0.505,0.504


In [41]:
print(best_features_dict)

{'initial': array(['Closeness_initial_median', 'Degrees_initial_median',
       'LocalEfficiency_initial_median'], dtype='<U31'), '2-level features': array(['Closeness_G_2click_1simplex_median', 'Closeness_initial_median',
       'Degrees_G_2click_1simplex_median', 'Degrees_initial_median',
       'GlobalEfficiency_G_2click_1simplex_median',
       'GlobalEfficiency_initial_median',
       'LocalEfficiency_G_2click_1simplex_median'], dtype='<U41'), '3-level features': array(['AvgNodeDegree_G_3click_2simplex_median',
       'Betweenness_G_3click_1simplex_median',
       'Betweenness_G_3click_2simplex_median',
       'Closeness_G_3click_2simplex_median',
       'Clustering_G_3click_1simplex_median',
       'Clustering_G_3click_2simplex_median', 'Degrees_initial_median',
       'GlobalEfficiency_G_3click_1simplex_median',
       'LocalEfficiency_G_3click_1simplex_median',
       'LocalEfficiency_initial_median', 'n_3click_2simplex_median'],
      dtype='<U41')}


In [21]:
from sklearn.model_selection import cross_val_score

In [22]:
def selection_step(model, X, y, used_features=(), cv=3):
    """
    Parameters
    ==========
        X: ndarray - training inputs
        y: ndarray - training targets
        used_features: - list of features
        cv: int - number of folds

    Returns
    =======
        scores - dictionary of scores
    """
    
    scores = {}
    
    ### BEGIN Solution
    scores = cross_val_score(model, X[used_features], y, scoring='roc_auc', cv=cv)
    ### END Solution

    return scores

In [23]:
def forward_steps(X, y, n_rounds, method):
    """
    Parameters
    ==========
        X: ndarray - training inputs
        y: ndarray - training targets
        n_rounds: int - early stop when score doesn't increase n_rounds
        method: sklearn model

    Returns
    =======
        feat_best_list - list of features
    """
    
    feat_best_list = []
    features_list = X.columns.values.tolist()
    prev_cv = np.nan
    is_first_iteration = True

    n_round = 0
    
    ### BEGIN Solution
    while n_round < n_rounds:
        n_round += 1
        
        if features_list == []:
            break
        
        current_best_cv = np.nan
        current_best_feature = np.nan
        
        for feature in features_list:
            
            curr_cv = selection_step(method, X, y, used_features= feat_best_list + [feature], cv=3).mean()
            
            if feature == features_list[0]:
                current_best_cv = curr_cv
                current_best_feature = feature
            elif curr_cv > current_best_cv:
                current_best_cv = curr_cv
                current_best_feature = feature
                
        features_list.remove(current_best_feature)
        feat_best_list.append(current_best_feature)
        
        if is_first_iteration: 
            prev_cv = current_best_cv
            is_first_iteration = False
        elif current_best_cv >  prev_cv:
            n_round = 0
            prev_cv = current_best_cv

    ### END Solution
    
    return feat_best_list


In [24]:
X = df[best_features_dict['3-level features']]
y = target

In [33]:
import warnings
warnings.filterwarnings('ignore')

from sklearn.tree import DecisionTreeClassifier
### BEGIN Solution
dtr = DecisionTreeClassifier()
n_rounds = 7
best_features = forward_steps(X, y, n_rounds, dtr)

print(best_features)
### END Solution

['LocalEfficiency_initial_median', 'GlobalEfficiency_G_3click_1simplex_median', 'Degrees_initial_median', 'Betweenness_G_3click_2simplex_median', 'Clustering_G_3click_2simplex_median', 'Closeness_G_3click_2simplex_median', 'Betweenness_G_3click_1simplex_median', 'Clustering_G_3click_1simplex_median', 'AvgNodeDegree_G_3click_2simplex_median', 'n_3click_2simplex_median', 'LocalEfficiency_G_3click_1simplex_median']


In [34]:
for method in methods_list:
    #best_features = forward_steps(X, y, n_rounds, method)
    score = cross_val_score(method, X=train_normed, y=target, cv=rk, scoring=scoring_method)
    print(score.mean())

0.5577461205461205
0.5436649045399046
0.56633947996448
0.49798187923187925


In [35]:
for method in methods_list:
    #best_features = forward_steps(X, y, n_rounds, method)
    score = cross_val_score(method, X=train_normed, y=target, cv=rk, scoring=scoring_method)
    print(score.mean())

0.5854537878787878
0.5625404512154513
0.5923399295149294
0.5075247641247641


In [27]:
method_scores.mean()

0.560134817959818